In [1]:
import csv
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import helpers as analysis
from numpy import linalg as LA
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abbyk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [7]:
df = pd.read_csv('data/data_cleaning_coffee.csv')
df

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,est_price_split
0,https://www.coffeereview.com/review/sweety-esp...,\r\n\r\n\r\n95\r\n\r\n\r\nA.R.C.\r\n“Sweety” ...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,$48.71/340 grams,November 2017,...,9.0,8.0,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"['HKD', '$250/227', 'grams']"
1,https://www.coffeereview.com/review/flora-blen...,\r\n\r\n\r\n94\r\n\r\n\r\nA.R.C.\r\nFlora Ble...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,$30.78/340 grams,November 2017,...,9.0,8.0,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","['HKD', '$158/227', 'grams']"
2,https://www.coffeereview.com/review/ethiopia-s...,\r\n\r\n\r\n92\r\n\r\n\r\nRevel Coffee\r\nEth...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/340 grams,November 2017,...,9.0,8.0,8.0,9.0,8.0,9.0,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","['$16.00/12', 'ounces']"
3,https://www.coffeereview.com/review/ethiopia-s...,\r\n\r\n\r\n92\r\n\r\n\r\nRoast House\r\nEthi...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/340 grams,November 2017,...,8.0,8.0,9.0,9.0,8.0,9.0,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,"['$19.00/16', 'ounces']"
4,https://www.coffeereview.com/review/ethiopia-g...,\r\n\r\n\r\n94\r\n\r\n\r\nBig Creek Coffee Ro...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/340 grams,November 2017,...,9.0,9.0,9.0,9.0,8.0,9.0,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,"['$16.50/12', 'ounces']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,https://www.coffeereview.com/review/finca-patz...,\r\n\r\n\r\n92\r\n\r\n\r\nEl Gran Cafe\r\nFin...,92,El Gran Cafe,Finca Patzibir,"Antigua, Guatemala","Lake Atitlán growing region, Guatemala",Medium-Light,$16.00/340 grams,November 2022,...,8.0,8.0,9.0,9.0,8.0,9.0,"Crisply sweet, nut-toned. Almond brittle, pie ...",Produced by Andres Fahsen of Finca Patzibar en...,"A quietly confident, sweetly nut-toned Guatema...","['$16.00/12', 'ounces']"
2273,https://www.coffeereview.com/review/proyecto-a...,\r\n\r\n\r\n93\r\n\r\n\r\nEl Gran Cafe\r\nPro...,93,El Gran Cafe,Proyecto Aurora,"Antigua, Guatemala","Antigua growing region, Guatemala",Medium-Light,$10.00/340 grams,November 2022,...,9.0,9.0,8.0,9.0,8.0,9.0,"Chocolaty, floral-framed. Cocoa nib, honeysuck...",Produced at Beneficio La Esperanza from trees ...,"A deeply floral, richly chocolaty Guatemala cu...","['$10.00/12', 'ounces']"
2274,https://www.coffeereview.com/review/finca-el-p...,\r\n\r\n\r\n93\r\n\r\n\r\nEl Gran Cafe\r\nFin...,93,El Gran Cafe,Finca El Potrero,"Antigua, Guatemala","Antigua growing region, Guatemala",Medium-Light,$10.00/340 grams,November 2022,...,9.0,9.0,8.0,9.0,8.0,9.0,"High-toned, enticingly sweet. Black cherry, na...",Produced by Josué Morales of Finca El Potrero...,"A bright, balanced, juicy Guatemala cup driven...","['$10.00/12', 'ounces']"
2275,https://www.coffeereview.com/review/chacaya-sa...,\r\n\r\n\r\n93\r\n\r\n\r\nEl Gran Cafe\r\nCha...,93,El Gran Cafe,Chacayá Santiago Atitlán,"Antigua, Guatemala","Lake Atitlán gr

In [8]:
combined_descriptions = df[["desc_1"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_names = df[["name"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_locs = df[["origin"]].apply(lambda x : ' '.join(x.dropna()), axis=1)

combined_descriptions = [x for x in combined_descriptions]
combined_names = [x + " from " + combined_locs[i] for i, x in enumerate(combined_names)]

vectorizer = TfidfVectorizer()


#Replace query with the user query here 
query = ["Nutty and Sweet"]
# query = query.lower()
# query = query.split(" ")

doc_vectors  = vectorizer.fit_transform(query + combined_descriptions).toarray()

index_to_vocab = {i:v for i, v in enumerate(vectorizer.get_feature_names_out())}
doc_to_index = {v:i for i, v in enumerate(combined_names)}
index_to_doc_descriptions = {i:{'name': v, 'description': combined_descriptions[i]}for i, v in enumerate(combined_names)}

cosineSims = (np.dot(doc_vectors[0], np.transpose(doc_vectors[1:]))/(LA.norm(doc_vectors[0])* LA.norm(doc_vectors[1:])))

cosineSims = [(x, i) for i, x in enumerate(cosineSims)]

cosineSims = sorted(cosineSims, key = lambda x:x[0], reverse = True)

topTen = cosineSims[:10]
answer  = []
for sim, index in topTen: 
  answer.append((index_to_doc_descriptions[index], sim))




AttributeError: 'TfidfVectorizer' object has no attribute 'get_feature_names_out'

In [ ]:
search_terms = "my query"
documents = ["my","list","of","docs"]

vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform([search_terms] + documents).toarray()

print(np.dot(doc_vectors[0], np.transpose(doc_vectors[1:]))/(LA.norm(doc_vectors[0])* LA.norm(doc_vectors[1:])))

# document_scores = [item.item() for item in cosine_similarities[1:]]

[0.31395688 0.         0.         0.        ]


In [ ]:
def findTopTen(user_query):
    """
    Takes a user query and returns an array of (dictionary, integer) pairs
    Dictionary is a dictionary of 'name', and 'description' of a coffee, and the integer is the cosine similarity

    Result is returned in order of cosine similarity highest to lowest
    User query must be input as a string
    """

    import csv
    import pandas as pd
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.stem import PorterStemmer
    from collections import Counter
    import string
    from sklearn.feature_extraction.text import TfidfVectorizer
    import helpers as analysis
    from numpy import linalg as LA
    import numpy as np

    df = pd.read_csv("data/data_cleaning_coffee.csv")

    combined_descriptions = df[["desc_1"]].apply(lambda x: " ".join(x.dropna()), axis=1)
    combined_names = df[["name"]].apply(lambda x: " ".join(x.dropna()), axis=1)
    combined_locs = df[["origin"]].apply(lambda x: " ".join(x.dropna()), axis=1)

    combined_descriptions = [x for x in combined_descriptions]
    combined_names = [
        x + " from " + combined_locs[i] for i, x in enumerate(combined_names)
    ]

    vectorizer = TfidfVectorizer()

    # Replace query with the user query here
    query = [user_query]

    doc_vectors = vectorizer.fit_transform(query + combined_descriptions).toarray()
    # don't think we need these, but in the event that we have a very slow query, we can use this
    # index_to_vocab = {i: v for i, v in enumerate(vectorizer.get_feature_names_out())}
    # doc_to_index = {v: i for i, v in enumerate(combined_names)}
    index_to_doc_descriptions = {
        i: {"name": v, "description": combined_descriptions[i]}
        for i, v in enumerate(combined_names)
    }

    cosineSims = np.dot(doc_vectors[0], np.transpose(doc_vectors[1:])) / (
        LA.norm(doc_vectors[0]) * LA.norm(doc_vectors[1:])
    )

    cosineSims = [(x, i) for i, x in enumerate(cosineSims)]

    cosineSims = sorted(cosineSims, key=lambda x: x[0], reverse=True)

    topTen = cosineSims[:10]
    answer = []
    for sim, index in topTen:
        answer.append((index_to_doc_descriptions[index], sim))

    return answer


In [ ]:
answer = findTopTen("sweet and floral")

for i, x in enumerate(answer): 
  print(x[0]['name'])
  print(x[0]['description'])
  print(x[1])


Finca Lerida Gesha Natural from Jaramillo, Boquete growing region, western Panama
Richly sweet, floral-toned. Magnolia, date, almond, caramel, fresh-cut cedar in aroma and cup. Sweet structure with gentle, round acidity; full, syrupy-smooth mouthfeel. Floral and nut-toned finish.
0.009640267566651109
Costa Rica Central Valley Brandy Ball from Central Valley, Costa Rica
Richly sweet, floral-toned. Freesia-like flowers, orange zest, hazelnut, caramel, cedar in aroma and cup. Sweet structure with gentle, round acidity; full, syrupy-smooth mouthfeel. Crisp floral- and nut-toned finish.
0.00933189866436275
Panama Geisha from Jaramillo, Boquete growing region, Panama
Richly sweet, floral-toned. Honeysuckle, raisin, pistachio, caramel, fresh-cut cedar in aroma and cup. Sweet-toned structure with gentle, round acidity; velvety-smooth mouthfeel. Floral and nut-toned finish.
0.008855532724805383
Honduras Relationship Coffee Norma Lara from Ocotepeque Department, Honduras
Richly floral-toned, dee